# Results

The following code generates choropleth maps showing the number of cyclable trips in São Paulo districts.

In [5]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2
import saopaulo.sp_grid as gr
import bikescience.distributions as dist
import saopaulo.cycling_potential as cp
import saopaulo.choropleth_folium as choro_folium
import saopaulo.choropleth as choro
from bikescience.slope import plot_slope, plot_slopes, split_route
from shapely.geometry import LineString, Point
import ast
from numpy import nan
import math
from bikescience.arrow import draw_arrow

import folium
import pandas as pd
import geopandas as gpd
from ipywidgets import interact_manual, widgets, fixed
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
import warnings
import numpy as np
warnings.simplefilter('ignore')

gr.SP_LAT = -23.63
gr.SP_LON = -46.55

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/pedro/.local/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [6]:
districts_name = [
    'Bela Vista', 'Bom Retiro', 'Cambuci', 'Consolação', 'Liberdade', 
    'República', 'Santa Cecília', 'Sé','Butantã', 'Morumbi', 'Raposo Tavares', 
    'Rio Pequeno', 'Vila Sônia', 'Barra Funda', 'Jaguara', 'Jaguaré', 'Lapa', 
    'Perdizes', 'Vila Leopoldina', 'Alto de Pinheiros', 'Itaim Bibi', 
    'Jardim Paulista', 'Pinheiros','Aricanduva', 'Carrão', 'Vila Formosa',
    'Água Rasa', 'Belém', 'Brás', 'Mooca', 'Pari', 'Tatuapé', 'Artur Alvim',
    'Cangaíba', 'Penha', 'Vila Matilde', 'São Lucas', 'Sapopemba',
    'Vila Prudente','Cidade Tiradentes', 'Ermelino Matarazzo', 'Ponte Rasa',
    'Guaianases', 'Lajeado', 'Itaim Paulista', 'Vila Curuçá', 'Cidade Líder',
    'Itaquera', 'José Bonifácio', 'Parque do Carmo', 'Iguatemi', 'São Mateus',
    'São Rafael', 'Jardim Helena', 'São Miguel', 'Vila Jacuí','Cachoeirinha', 
    'Casa Verde', 'Limão', 'Brasilândia', 'Freguesia do Ó', 'Anhanguera', 
    'Perus', 'Jaraguá', 'Pirituba', 'São Domingos','Jaçanã', 'Tremembé', 
    'Mandaqui', 'Santana', 'Tucuruvi', 'Vila Guilherme', 'Vila Maria', 
    'Vila Medeiros','Cursino', 'Ipiranga', 'Sacomã', 'Jabaquara', 'Moema', 
    'Saúde', 'Vila Mariana', 'Campo Limpo', 'Capão Redondo', 'Vila Andrade', 
    'Cidade Dutra', 'Grajaú', 'Socorro', 'Cidade Ademar', 'Pedreira', 
    'Jardim Ângela', 'Jardim São Luís', 'Marsilac', 'Parelheiros',
    'Campo Belo', 'Campo Grande', 'Santo Amaro']

In [7]:
#read data

districts = True
OD_zone = !districts 

if districts:
    zone_shp = gpd.read_file('../data/sao-paulo/od/shapes/Distritos_2017_region.shp')
    zone_shp.crs = {'init': 'epsg:31983'}  
    zone_shp.to_crs(epsg='4326', inplace=True)
    # converting to km^2
    zone_shp['Area_ha_2'] = zone_shp['Area_ha'] / 100
    zone_shp['NumeroZona'] = zone_shp['NumeroDist']
    zone_shp['NomeZona'] = zone_shp['NomeDistri']
    zone_shp['NumeroMuni'] = 0

    numeroMuni = []
    for _, d in zone_shp.iterrows():
        if d['NomeDistri'] in districts_name:
            numeroMuni.append(36)
        else:
            numeroMuni.append(0)
    zone_shp['NumeroMuni'] = numeroMuni
    
else:
    zone_shp = gpd.read_file('../data/sao-paulo/od/shapes/Zonas_2017_region.shp')
    zone_shp.crs = {'init': 'epsg:31983'}  
    zone_shp.to_crs(epsg='4326', inplace=True)

display(zone_shp)

# read routes files and join with general trips file
od_trips = pd.read_csv('bases/complete_csv.csv')
od_trips['final_potential_v2'] = (od_trips['inclination_potential'] + od_trips['distance_potential']) / 2
# fix FE_VIA with wrong format (because of thousand separator)
od_trips.loc[od_trips['FE_VIA'] >= 10000, 'FE_VIA'] = od_trips.loc[od_trips['FE_VIA'] >= 10000, 'FE_VIA'] / 1000

#od_trips.set_index('ID_ORDEM', inplace=True, drop=False)

od_trips.dropna(inplace = True)

the_grid = gr.create(n=9, west_offset=-0.15, east_offset=0.23, north_offset=0.19, south_offset=-0.46)

,NumeroDist,NomeDistri,Area_ha,geometry,Area_ha_2,NumeroZona,NomeZona,NumeroMuni
0,1,Água Rasa,715.05,"POLYGON ((-46.55380 -23.56843, -46.55387 -23.5...",7.1505,1,Água Rasa,36
1,2,Alto de Pinheiros,751.26,"POLYGON ((-46.69209 -23.54631, -46.69257 -23.5...",7.5126,2,Alto de Pinheiros,36
2,3,Anhanguera,3339.95,"POLYGON ((-46.76082 -23.42733, -46.76088 -23.4...",33.3995,3,Anhanguera,36
3,4,Aricanduva,686.26,"POLYGON ((-46.50153 -23.57941, -46.50154 -23.5...",6.8626,4,Aricanduva,36
4,5,Artur Alvim,653.04,"POLYGON ((-46.47300 -23.54029, -46.47302 -23.5...",6.5304,5,Artur Alvim,36
...,...,...,...,...,...,...,...,...
129,130,São Caetano do Sul,1536.53,"POLYGON ((-46.58317 -23.64088, -46.58326 -23.6...",15.3653,130,São Caetano do Sul,0
130,131,São Lourenço da Serra,18674.34,"POLYGON ((-46.98852 -23.79984, -46.98847 -23.7...",186.7434,131,São Lourenço da Serra,0
131,132,Suzano,20666.94,"POLYGON ((-46.38321 -23.61775, -46.38318 -23.6...",206.6694,132,Suzano,0
132,133,Taboão da Serra,2043.37,"POLYGON ((-46.81819 -23.63701, -46.82138 -23.6...",20.4337,133,Taboão da Serra,0


In [8]:
od_trips = gpd.GeoDataFrame(od_trips, crs={'init': 'epsg:4326'},
                 geometry=[LineString(eval(r)) for r in od_trips['route']])

In [9]:

print('------------------------------')
print('--------- ALL TRIPS ----------')
print('------------------------------')
print('')

print('Trips:                             ', 
      round(sum(od_trips['FE_VIA'])))

print('Trips (filtering age):             ', 
      round(sum(od_trips
                .loc[od_trips['IDADE'] > 16]
                .loc[od_trips['IDADE'] < 65]['FE_VIA'])))

print('Cyclable Trips:                     ', 
      round(sum(od_trips.loc[od_trips['final_potential_v2'] > .75]['FE_VIA'])))

print('Cyclable Trips (filtering age):     ', 
      round(sum(od_trips
                .loc[od_trips['IDADE'] > 16]
                .loc[od_trips['IDADE'] < 65]
                .loc[od_trips['final_potential_v2'] > .75]['FE_VIA'])))


#print('Total de viagens cicláveis    (> 0.7):', 
#      round(sum(od_trips.loc[od_trips['final_potential'] > .7]['FE_VIA'])))

print('')
print('')
print('------------------------------')
print('----------- CAR --------------')
print('------------------------------')
print('')
print('Trips:                              ', 
      round(sum(od_trips
                .loc[od_trips['modal'] == 'car']['FE_VIA'])))

print('Trips (filtering age):               ', 
      round(sum(od_trips 
                .loc[od_trips['IDADE'] > 16]
                .loc[od_trips['IDADE'] < 65]
                .loc[od_trips['modal'] == 'car']['FE_VIA'])))


#print('Total de viagens cicláveis    (carro) (> 0.7):', 
#      round(sum(od_trips.loc[od_trips['modal'] == 'car'].loc[od_trips['final_potential'] > .7]['FE_VIA'])))

print('Cyclable Trips:                      ', 
      round(sum(od_trips
                .loc[od_trips['modal'] == 'car']
                .loc[od_trips['final_potential_v2'] > .75]['FE_VIA'])))

print('Cyclable Trips (filtering age):      ', 
      round(sum(od_trips
                .loc[od_trips['IDADE'] > 16]
                .loc[od_trips['IDADE'] < 65]
                .loc[od_trips['modal'] == 'car']
                .loc[od_trips['final_potential_v2'] > .75]['FE_VIA'])))

------------------------------
--------- ALL TRIPS ----------
------------------------------

Trips:                              47477291
Trips (filtering age):              35590335
Cyclable Trips:                      7225520
Cyclable Trips (filtering age):      4380353


------------------------------
----------- CAR --------------
------------------------------

Trips:                               11582848
Trips (filtering age):                9149911
Cyclable Trips:                       1670990
Cyclable Trips (filtering age):       1150982


In [10]:
# calculate the number of trips that passes through 
def calculate_trips_zone_intersection (trips, zones, reference):
    
    trips_geometries = list(trips['geometry'])
    trips_expansion = list(trips['FE_VIA'])

    trips_per_zone = [0]*len(zone_shp) # hash to store trips indexed by zones

    progress = 0
    
    for z in range(len(zones)):
        zone = zones.iloc[z]['geometry']
        
        if z / len(zones) >= progress:
            print (round(progress * 100), '%')
            progress += 0.25
            
        for i in range (len (trips_geometries)):
            if reference == 'intersection':
                if (trips_geometries[i].intersects(zone)):
                    trips_per_zone[z] += trips_expansion[i]
            else:
                p = trips_geometries[i].coords[reference]
                if (Point(p[0], p[1]).within(zone)):
                    trips_per_zone[z] += trips_expansion[i]
    return trips_per_zone   

In [11]:
def filter_potential_widgets(btn_text, process_function):
    
    width = '200px'
    layout = widgets.Layout(width = '500px')
    
    im = interact_manual(
        process_function,
        final_potential = widgets.FloatRangeSlider(
            value=[0.8, +1.],
            min=0, max=1., step=0.05,
            description='Cycling Potential:\t',
            style={'description_width': width},
            layout=layout
        ),
        distance_potential = widgets.FloatRangeSlider(
            value=[0, +1.],
            min=0, max=1., step=0.05,
            description='Potential (distance):\t',
            style={'description_width': width},
            layout=layout
        ),
        inclination_potential = widgets.FloatRangeSlider(
            value=[0, +1.],
            min=0, max=1., step=0.05,
            description='Potential (slope):\t',
            style={'description_width': width},
            layout=layout
        ),
       # age_potential = widgets.FloatRangeSlider(
       #     value=[0, +1.],
       #     min=0, max=1., step=0.05,
       #     description='Age Potential:\t',
       #     style={'description_width': width},
       #     layout=layout
       # ),
        distance = widgets.FloatRangeSlider(
            value=[2, 50],
            min=0, max=50, step=0.05,
            description='Distance (km):\t',
            style={'description_width': width},
            layout=layout
        ),
        
        modal = widgets.Dropdown(options = [('All', None), ('Car', 'car'), ('Subway', 'subway'),
                                            ('Pedestrian', 'pedestrian'), ('Motorcycle', 'motorcycle'),
                                            ('Train', 'train')],
                                 description = 'Transport Modal', 
                                 style={'description_width': width}),
        reference = widgets.Dropdown(options = [('Whole Trip', 'intersection'),('Origin', 0), 
                                                ('Destination', -1)],
                                     description = 'Trip Reference:\t', 
                                     style={'description_width': width}),
        
        density = widgets.Checkbox(description = 'Density'),
        debug = widgets.Checkbox(description = 'Show trips'),
        plot_rmsp = widgets.Checkbox(description = 'Metropolitan Area'),
        title = widgets.Text(description = 'Map Title', style={'description_width': width}),
    )
    im.widget.children[10].description = 'Generate Map'
    
def inside_limits (limits, value):
    return value >= limits[0] and value <= limits[1]
    
def filter_potential(final_potential, distance_potential, 
                     #age_potential, 
                     inclination_potential, distance, modal):
    if modal != None:
        print(modal)
        aux = od_trips.loc[od_trips['modal'] == modal]
    else:
        aux = od_trips
    
    #aux = aux.loc[aux['age_potential'] >= age_potential[0]]
    #aux = aux.loc[aux['age_potential'] <= age_potential[1]]
    aux = aux.loc[aux['distance_potential'] >= distance_potential[0]]
    aux = aux.loc[aux['distance_potential'] <= distance_potential[1]]
    aux = aux.loc[aux['inclination_potential'] >= inclination_potential[0]]
    aux = aux.loc[aux['inclination_potential'] <= inclination_potential[1]]
    aux = aux.loc[aux['final_potential_v2'] >= final_potential[0]]
    aux = aux.loc[aux['final_potential_v2'] <= final_potential[1]]
    aux = aux.loc[aux['length'] >= distance[0] * 1000]
    aux = aux.loc[aux['length'] <= distance[1] * 1000]
    return aux

In [12]:
def plot_potential_trips_per_zone(final_potential, distance_potential, inclination_potential, 
                                  #age_potential, 
                                  distance,
                                  modal, reference, 
                                  debug=False, plot_rmsp = False, density = False,
                                  title = None):
    df = filter_potential(final_potential, distance_potential, inclination_potential, distance, modal)
    #df = filter_potential(final_potential, distance_potential, age_potential, inclination_potential, modal)
    
    global global_df 
    global_df = df
    zones = zone_shp.loc[zone_shp["NumeroMuni"] == 36] if not plot_rmsp else zone_shp
    
    trips_per_zone = calculate_trips_zone_intersection(df, zones, reference)
    
    if density:
        df_total = od_trips if modal == None else od_trips.loc[od_trips['modal'] == modal]
        total_per_zone = calculate_trips_zone_intersection(df_total, zones, reference)
        perc = [0 if total_per_zone[i] == 0 else trips_per_zone[i] / total_per_zone[i] * 100
                for i in range(len(zone_shp))]
        zone_shp['potential_trips'] = perc
        
    else:
        zone_shp['potential_trips'] = trips_per_zone

    # precisa filtrar de novo pra plotar
    zones = zone_shp.loc[zone_shp["NumeroMuni"] == 36] if not plot_rmsp else zone_shp
    
    tooltip_columns = ['NomeZona', 'potential_trips']
    tooltip_aliases = ['District', 'Potential Trips']
    fmap = gr.map_around_sp(the_grid=None,zoom=10,plot_grid=False)
    choro_folium.plot_choropleth(fmap, title, 'YlOrBr', lambda x : x['potential_trips'], 
                          zones, tooltip_columns, tooltip_aliases, plot_rmsp)
    
    if debug:
        #display(df)
        plot_slopes (fmap, df['geometry'], 1000, False)
    fmap.save('maps/' + title + '_potential.html')
    display(fmap)


In [13]:
filter_potential_widgets('Filter trips', plot_potential_trips_per_zone)

interactive(children=(FloatRangeSlider(value=(0.8, 1.0), description='Cycling Potential:\t', layout=Layout(wid…